In [1]:
import numpy as np
from sklearn import preprocessing
import nltk
import csv
import pandas as pd


import networkx as nx
import community

import random
from sklearn import svm
from sklearn.metrics.pairwise import linear_kernel
from sklearn import preprocessing
import nltk
import csv
from bs4 import BeautifulSoup

In [72]:
file = "Period1.csv"
file2 = "Period2.csv"
file3 = "TestData.csv"

def readfile(filename):
    with open(filename, "r") as f:
        reader = csv.reader(f)
        data_set = list(reader)
        
        data_set = [[element[0].split(" ") for element in data_set],[element[1].split(" ") for element in data_set],[element[2].split(" ") for element in data_set]]
        del data_set[0][0]
        del data_set[1][0]
        del data_set[2][0]
        return data_set

In [74]:
data =readfile(file)
data2 = readfile(file2)
testdata = readfile(file3)

In [4]:
def create_features(data):
    features = pd.DataFrame([[data[1][i],data[2][i]]for i in range(len(data[0]))])
    features.columns = ['From', 'To']
                    
    year = [i[0] for i in data] 

    features['From'] = features['From'].apply(lambda x:x[0])
    features['To'] = features['To'].apply(lambda x:x[0])
    
    return features

In [5]:
features = create_features(data)
features2 = create_features(data2)

In [32]:
def create_graph(data):
    G = nx.Graph()
    
    linked_nodes = [(data['From'][i],data['To'][i])for i in range(len(data))]
    
    G.add_edges_from(linked_nodes)
    
    return G

def create_digraph(data):
    G = nx.DiGraph()
    
    linked_nodes = [(data['From'][i],data['To'][i])for i in range(len(data))]
    
    G.add_edges_from(linked_nodes)
    
    return G

def labeling(features,features2):
    label = []
    for i in range(features.shape[0]):
        a = features['From'][i]
        b = features['To'][i]
        if(a in features2.values or b in features2.values):
            label.append(1)
        else:
            label.append(0)
    return label

In [11]:
graph = create_graph(features)
graph2 = create_digraph(features)
nx.is_directed(graph2)

True

In [8]:
def common_neighbors(features, G):
    nb_common_neighbors = []
    for i in range(features.shape[0]):
        a = features['From'][i]
        b = features['To'][i]
        nb_common_neighbors.append(len(sorted(nx.common_neighbors(G, a, b)))) # ajoute le nombre de voisins communs
    return nb_common_neighbors

def Jaccard_coef(features, G):
    J = []
    for i in range(features.shape[0]):
        a = features['From'][i]
        b = features['To'][i]
        pred = nx.jaccard_coefficient(G,[(a,b)])
        for u,v,p in pred:
            J.append(p)
    return J

def betweenness_diff(features, G):
    btw = nx.betweenness_centrality(G, 50)
    btw_diff = []
    for i in range(features.shape[0]):
        a = features['From'][i]
        b = features['To'][i]
        btw_diff.append(btw[b] - btw[a])
    return btw_diff

def in_link_diff(features, G2):
    diff = []
    for i in range(features.shape[0]):
        a = features['From'][i]
        b = features['To'][i]
        diff.append(len(G2.in_edges(b)) - len(G2.in_edges(a)))
    return diff

def is_same_cluster(partition, features):
    same_cluster = []
    for i in range(features.shape[0]):
        a = features['From'][i]
        b = features['To'][i]
        if(partition[a] == partition[b]):
            same_cluster.append(1)
        else:
            same_cluster.append(0)
    return same_cluster

In [17]:
no_common_neighbors = common_neighbors(features, graph)
features['No_common_neighbors'] = no_common_neighbors

Jaccard = Jaccard_coef(features, graph)
features['Jaccard_coef'] = Jaccard

btw_diff = betweenness_diff(features, graph)
features['Betweenness_diff'] = btw_diff



In [20]:
diff = in_link_diff(features, graph2)
features['In_link_diff'] = diff

partition = community.best_partition(graph)
same_cluster_train = is_same_cluster(partition, features)
features['Is_same_cluster'] = same_cluster_train

# result_list.iloc[0:10000].to_csv('result.csv', sep=',')
features.to_csv('graph_features_training.csv', sep=',')

In [23]:
#testing
graph_test = create_graph(features2)
graph_test_2 = create_digraph(features2)

no_common_neighbors_test = common_neighbors(features2, graph_test)
print(len(no_common_neighbors_test))
features2['No_common_neighbors'] = no_common_neighbors_test

Jaccard_test = Jaccard_coef(features2, graph_test)
features2['Jaccard_coef'] = Jaccard_test

btw_diff_test = betweenness_diff(features2, graph_test)
features2['Betweenness_diff'] = btw_diff_test



98353


In [21]:


diff2 = in_link_diff(features2, graph_test_2)
features2['In_link_diff'] = diff2

partition2 = community.best_partition(graph_test)
same_cluster_test = is_same_cluster(partition2, features2)
features2['Is_same_cluster'] = same_cluster_test

# result_list.iloc[0:10000].to_csv('result.csv', sep=',')
features2.to_csv('graph_features_testing.csv', sep=',')

In [45]:
# features2['label'] = np.nan


In [41]:
features2.to_csv('graph_features_testing.csv', sep=',')

In [33]:
label = labeling(features, features2)
features['label'] = label

In [44]:
features.to_csv('graph_features_training_with_label.csv', sep=',')

In [46]:
#prediction

train = pd.read_csv('graph_features_training.csv')
del train['Unnamed: 0']


In [47]:
#training
X_train = train.as_matrix()

y_train = label

#testing
test = pd.read_csv('graph_features_testing.csv')
del test['Unnamed: 0']

X_test = test.as_matrix()


In [38]:
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier, ExtraTreesClassifier)
from sklearn import cross_validation
from sklearn.ensemble import (RandomTreesEmbedding, RandomForestClassifier,
                              GradientBoostingClassifier, ExtraTreesClassifier)
import xgboost as xgb

In [48]:
# SVM

clf = ExtraTreesClassifier(max_features=None, min_samples_leaf=20, n_estimators = 500, n_jobs=3)

clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [50]:
# extra trees classifier

clf = ExtraTreesClassifier(max_features=None,min_samples_leaf=10,n_estimators=500,n_jobs=3)
cv = cross_validation.cross_val_score(clf, X_train, y_train, cv=5)

In [52]:
#xg boost classifier
# 1st tuning

gbm = xgb.XGBClassifier(max_depth=6, n_estimators=500, learning_rate=0.01)
cv = cross_validation.cross_val_score(gbm, X_train, y_train, cv=5)
print(np.mean(cv))


0.93311633282


In [53]:
# 2nd tuning
gbm = xgb.XGBClassifier(max_depth=4, n_estimators=500, learning_rate=0.05)
gbm.fit(X_train, y_train)
pred = gbm.predict(X_test)

In [54]:
#grid search
from sklearn.grid_search import GridSearchCV
parameters = {'n_estimators':[500,1000],
        'learning_rate': [0.05, 0.01, 0.001]}

clf = GridSearchCV( xgb.XGBClassifier(max_depth=4), parameters, n_jobs=4, cv=5, verbose = 10)
clf.fit(X_train, y_train)

/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] learning_rate=0.05, n_estimators=500 ............................
[CV] learning_rate=0.05, n_estimators=500 ............................
[CV] learning_rate=0.05, n_estimators=500 ............................
[CV] learning_rate=0.05, n_estimators=500 ............................
[CV] ... learning_rate=0.05, n_estimators=500, score=0.934511 -  35.6s
[CV] learning_rate=0.05, n_estimators=500 ............................
[CV] ... learning_rate=0.05, n_estimators=500, score=0.933738 -  35.7s
[CV] learning_rate=0.05, n_estimators=1000 ...........................
[CV] ... learning_rate=0.05, n_estimators=500, score=0.935706 -  35.7s
[CV] learning_rate=0.05, n_estimators=1000 ...........................
[CV] ... learning_rate=0.05, n_estimators=500, score=0.934802 -  35.8s
[CV] learning_rate=0.05, n_estimators=1000 ...........................
[CV] ... learning_rate=0.05, n_estimators=500, score=0.934188 -  35.9s
[CV] learning_rat

[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  1.2min


[CV] .. learning_rate=0.05, n_estimators=1000, score=0.935706 - 1.1min
[CV] learning_rate=0.05, n_estimators=1000 ...........................
[CV] .. learning_rate=0.05, n_estimators=1000, score=0.935318 - 1.2min
[CV] learning_rate=0.01, n_estimators=500 ............................
[CV] .. learning_rate=0.05, n_estimators=1000, score=0.934126 - 1.2min
[CV] learning_rate=0.01, n_estimators=500 ............................
[CV] .. learning_rate=0.05, n_estimators=1000, score=0.937030 - 1.1min
[CV] learning_rate=0.01, n_estimators=500 ............................
[CV] ... learning_rate=0.01, n_estimators=500, score=0.932089 -  36.2s
[CV] learning_rate=0.01, n_estimators=500 ............................
[CV] ... learning_rate=0.01, n_estimators=500, score=0.931607 -  36.4s
[CV] learning_rate=0.01, n_estimators=500 ............................


[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:  2.4min


...

[CV] .. learning_rate=0.01, n_estimators=1000, score=0.933252 - 1.2min
[CV] learning_rate=0.001, n_estimators=500 ...........................
[CV] .. learning_rate=0.01, n_estimators=1000, score=0.934543 - 1.2min
[CV] learning_rate=0.001, n_estimators=500 ...........................
[CV] .. learning_rate=0.001, n_estimators=500, score=0.928410 -  36.2s
[CV] learning_rate=0.001, n_estimators=500 ...........................
[CV] .. learning_rate=0.001, n_estimators=500, score=0.928408 -  35.9s
[CV] learning_rate=0.001, n_estimators=500 ...........................
[CV] .. learning_rate=0.001, n_estimators=500, score=0.928408 -  36.3s
[CV] learning_rate=0.001, n_estimators=1000 ..........................
[CV] .. learning_rate=0.01, n_estimators=1000, score=0.932961 - 1.2min
[CV] learning_rate=0.001, n_estimators=1000 ..........................
[CV] .. learning_rate=0.001, n_estimators=500, score=0.928408 -  38.2s
[CV] learning_rate=0.001, n_estimators=1000 ..........................
[CV] .

[Parallel(n_jobs=4)]: Done  27 out of  30 | elapsed:  6.4min remaining:   43.0s


[CV] . learning_rate=0.001, n_estimators=1000, score=0.928408 - 1.2min
[CV] . learning_rate=0.001, n_estimators=1000, score=0.928408 - 1.2min
[CV] . learning_rate=0.001, n_estimators=1000, score=0.928408 - 1.3min


[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:  7.3min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params={}, iid=True, n_jobs=4,
       param_grid={'n_estimators': [500, 1000], 'learning_rate': [0.05, 0.01, 0.001]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=10)

In [58]:
#submission
predictions = list(pred) 
predictions = zip(range(len(pred)), predictions)
with open("result_pred.csv","w") as pred1:
    csv_out = csv.writer(pred1)
    csv_out.writerow(["ID", "category"])
    for row in predictions:
        csv_out.writerow(row)

In [62]:
with open("result_pred.csv", "r") as f:
    reader = csv.reader(f)
    data_set = list(reader)
    data_set = [[element[0].split(" ") for element in data_set],[element[1].split(" ") for element in data_set]]  
    



# result_list = result_list.apply(lambda x:x[0])


In [70]:
result = pd.DataFrame(data_set[1][i] for i in range(1,len(data_set[0])))
result.index = np.arange(1, len(result) + 1)
result.index.name = 'target id'
result.columns = ['label']
# features["link"] = np.nan

# result['target id'] = result['target id'].apply(lambda x:x[0])
# result['label'] = result['label'].apply(lambda x:x[0])



In [71]:

result.iloc[0:10000].to_csv('result_pred_1.csv', sep=',')


In [65]:
print(len(features2))

98353


In [75]:
#TestData
features_test = create_features(testdata)


In [77]:
#testing
graph_testdata = create_graph(features_test)
graph_testdata_2 = create_digraph(features_test)

no_common_neighbors_testdata = common_neighbors(features_test, graph_testdata)
features_test['No_common_neighbors'] = no_common_neighbors_testdata

Jaccard_testdata = Jaccard_coef(features_test, graph_testdata)
features_test['Jaccard_coef'] = Jaccard_testdata

btw_diff_testdata = betweenness_diff(features_test, graph_testdata)
features_test['Betweenness_diff'] = btw_diff_testdata


diff3 = in_link_diff(features_test, graph_testdata_2)
features_test['In_link_diff'] = diff3

partition3 = community.best_partition(graph_testdata)
same_cluster_testdata = is_same_cluster(partition3, features_test)
features_test['Is_same_cluster'] = same_cluster_testdata

# result_list.iloc[0:10000].to_csv('result.csv', sep=',')
features_test.to_csv('graph_features_testing_data.csv', sep=',')

In [78]:
#testing
testing = pd.read_csv('graph_features_testing_data.csv')
del testing['Unnamed: 0']

X_test2 = testing.as_matrix()

In [79]:
pred_test = clf.predict(X_test2)

In [80]:
pred_test = gbm.predict(X_test2)

In [81]:
#submission
predictions = list(pred_test) 
predictions = zip(range(len(pred_test)), predictions)
with open("result_pred_2.csv","w") as pred1:
    csv_out = csv.writer(pred1)
    csv_out.writerow(["ID", "category"])
    for row in predictions:
        csv_out.writerow(row)

In [82]:
with open("result_pred_2.csv", "r") as f:
    reader = csv.reader(f)
    data_set = list(reader)
    data_set = [[element[0].split(" ") for element in data_set],[element[1].split(" ") for element in data_set]]  
    
result = pd.DataFrame(data_set[1][i] for i in range(1,len(data_set[0])))
result.index = np.arange(1, len(result) + 1)
result.index.name = 'target id'
result.columns = ['label']
result.iloc[0:10000].to_csv('result_pred_2.csv', sep=',')
